# Практическая работа №3. Свёрточные нейронные сети

**Работу выполнила:**

Алексеева Влада Вадимовна, ИТМО ID 367801

# Классификация цветов с помощью свёрточных нейронных сетей.


В работе необходимо познакомится с различными архитектурами сверхточных нейронных сетей и их обучением на GPU (англ. graphics processing, графический процессор) на языке программирования Python 3 и фреймворка Torch (PyTorch).  Для этого предлагается использовать ресурсы Google Colab - Colaboratory, для выполнения вычислений на GPU. После с ознакомления, выполнить практическое задане в конце данной тетради (notebook).

Рассмотрим [Датасет](https://www.kaggle.com/alxmamaev/flowers-recognition ) содержащий 4242 изображения цветов размеченных по 5 видам (тюльпан, ромашка, подсолнух, роза, одуванчик). Данный набор данных можно скачать по [ссылке](https://www.kaggle.com/alxmamaev/flowers-recognition ) с сайте kaggle.

Загрузите папку с картинками на гугл диск, чтобы не загружать ее каждый раз заново при перезапуске колаба. Структура файлов (можно посмотреть в меню слева) может быть такой: '/content/drive/My Drive/data/flowers'.

Обязательно подключите аппаратный ускоритель (GPU) к среде выполнения. В меню сверху: Среда выполнения -> Сменить среду выполнения

Первым делом разберите более детально код выполнив код ниже.

# Подготовка

Загружаем библиотеки. Фиксируем random.seed для воспроизводимости

In [5]:
import numpy as np
import os
import torch
import torchvision
from torchvision.datasets.utils import download_url
from torch.utils.data import random_split
from torchvision.datasets import ImageFolder
from torchvision import transforms
from torchvision.transforms import ToTensor
from torch.utils.data.dataloader import DataLoader
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
import random
from tqdm import tqdm
import matplotlib.pyplot as plt

random.seed(0)
torch.manual_seed(0)

Выбираем на чем будем делать вычисления - CPU или GPU (cuda)

In [6]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


Блок для соединения с Google Colab

In [7]:
from google.colab import drive

drive.mount('/content/drive', force_remount=True)

FOLDERNAME = 'Datas'

assert FOLDERNAME is not None, '[!] Enter the foldername.'

Mounted at /content/drive


In [8]:
prepare_imgs = torchvision.transforms.Compose(
    [
        torchvision.transforms.Resize((224, 224)), # Приводим картинки к одному размеру
        torchvision.transforms.ToTensor(), # Упаковывем их в тензор
        torchvision.transforms.Normalize(
            mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225] # Нормализуем картинки по каналам
        ),
    ]
)
# Задаем датасет
dataset = ImageFolder('/content/drive/My Drive/Datas', transform=prepare_imgs)

In [9]:
dataset.imgs[2]

('/content/drive/My Drive/Datas/Daisy/10172379554_b296050f82_n.jpg', 0)

In [10]:
class ValueMeter(object):
  """
  Вспомогательный класс, чтобы отслеживать loss и метрику
  """
  def __init__(self):
      self.sum = 0
      self.total = 0

  def add(self, value, n):
      self.sum += value*n
      self.total += n

  def value(self):
      return self.sum/self.total

def log(mode, epoch, loss_meter, accuracy_meter, best_perf=None):
  """
  Вспомогательная функция, чтобы
  """
  print(
      f'[{mode}] Epoch: {epoch:0.2f}. '
      f'Loss: {loss_meter.value():.2f}. '
      f'Accuracy: {100 * accuracy_meter.value():.2f}% ', end='\n')


# Сверточная нейросеть с нуля

## Вручную прописываем слои

**Как работает блок 'Сверточная нейросеть с нуля'? Как можно описать сверточный и пулинговый слой?**

Этот блок определяет простую последовательную архитектуру CNN с помощью `nn.Sequential`.

*   **Сверточный слой (`nn.Conv2d`):** Это основной строительный блок CNN. Он применяет набор обучаемых фильтров (ядер) к входному изображению. Каждый фильтр скользит по изображению, вычисляя скалярное произведение между своими весами и локальным участком изображения. Результат — карта признаков (feature map), которая активируется, если на изображении присутствует признак, за который 'отвечает' этот фильтр (например, вертикальный край). Параметры: `in_channels` (входные каналы), `out_channels` (количество фильтров), `kernel_size` (размер фильтра), `stride` (шаг), `padding` (отступ).

*   **Пулинговый слой (`nn.MaxPool2d`):** Этот слой выполняет даунсэмплинг (уменьшение пространственного размера) карт признаков. `MaxPool2d` выбирает максимальное значение из каждой небольшой локальной области (например, 2x2). Это помогает:
    1. Уменьшить количество параметров и вычислений в сети.
    2. Сделать представление признаков немного инвариантным к небольшим сдвигам и искажениям.
    3. Сжать информацию, оставляя наиболее значимые активации.

Архитектура работает по принципу: чередование сверточных слоев (для извлечения признаков) и пулинговых слоев (для уменьшения размерности), после чего данные 'выравниваются' (`Flatten`) и подаются на полносвязные слои (`Linear`) для окончательной классификации.

In [11]:
model = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2), # Выход: 64 x 16 x 16

            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.Conv2d(128, 128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2), # Выход: 128 x 8 x 8

            nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2), # Выход: 256 x 4 x 4

            nn.Flatten(),
            nn.Linear(256 * 28 * 28, 1024),
            nn.ReLU(),
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Linear(512, 5))

model.to(device) # Отправляем модель на девайс (GPU)

Sequential(
  (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): ReLU()
  (2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (3): ReLU()
  (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (6): ReLU()
  (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (8): ReLU()
  (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (11): ReLU()
  (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (13): ReLU()
  (14): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (15): Flatten(start_dim=1, end_dim=-1)
  (16): Linear(in_features=200704, out_features=1024, bias=True)
  (17): ReLU()
  (18): Linear(in_features=1024, out_features=512, bias=True)
  (19): ReLU()
  (20): Lin

# Задаем параметры и функцию для обучения. Разбиваем датасет на train/validation

In [12]:
batch_size = 32
optimizer = torch.optim.Adam(params = model.parameters())
lr = 0.001

Разбиваем датасет на train и validation

Задаем dataloader'ы - объекты для итеративной загрузки данных и лейблов для обучения и валидации

In [13]:
train_set, val_set = torch.utils.data.random_split(dataset, [len(dataset)-1000, 1000])
print('Размер обучающего и валидационного датасета: ', len(train_set), len(val_set))

loaders = {'training': DataLoader(train_set, batch_size, pin_memory=True,num_workers=2, shuffle=True),
           'validation':DataLoader(val_set, batch_size, pin_memory=True,num_workers=2, shuffle=False)}

Размер обучающего и валидационного датасета:  3352 1000


Функция для подсчета Accuracy

In [14]:
def accuracy(outputs, labels):
    _, preds = torch.max(outputs, dim=1)
    return torch.tensor(torch.sum(preds == labels).item() / len(preds))

Функция для обучения и валидации модели

In [15]:
def trainval(model, loaders, optimizer, epochs=10):
    """
    model: модель, которую собираемся обучать
    loaders: dict с dataloader'ами для обучения и валидации
    """
    loss_meter = {'training': ValueMeter(), 'validation': ValueMeter()}
    accuracy_meter = {'training': ValueMeter(), 'validation': ValueMeter()}

    loss_track = {'training': [], 'validation': []}
    accuracy_track = {'training': [], 'validation': []}

    for epoch in range(epochs): # Итерации по эпохам
        for mode in ['training', 'validation']: # Обучение - валидация
            # Считаем градиаент только при обучении:
            with torch.set_grad_enabled(mode == 'training'):
                # В зависимоти от фазы переводим модель в нужный ружим:
                model.train() if mode == 'training' else model.eval()
                for imgs, labels in tqdm(loaders[mode]):
                    imgs = imgs.to(device) # Отправляем тензор на GPU
                    labels = labels.to(device)
                    bs = labels.shape[0]  # Размер батча (отличается для последнего батча в лоадере)

                    preds = model(imgs) # Forward pass - прогоняем тензор с картинками через модель
                    loss = F.cross_entropy(preds, labels) # Считаем функцию потерь
                    acc = accuracy(preds, labels) # Считаем метрику

                    # Храним loss и accuracy для батча
                    loss_meter[mode].add(loss.item(), bs)
                    accuracy_meter[mode].add(acc, bs)

                    # Если мы в фазе обучения
                    if mode == 'training':
                        optimizer.zero_grad() # Обнуляем прошлый градиент
                        loss.backward() # Делаем backward pass (считаем градиент)
                        optimizer.step() # Обновляем веса

            # В конце фазы выводим значения loss и accuracy
            log(mode, epoch, loss_meter[mode], accuracy_meter[mode])

            # Сохраняем результаты по всем эпохам
            loss_track[mode].append(loss_meter[mode].value())
            accuracy_track[mode].append(accuracy_meter[mode].value())

    return loss_track, accuracy_track

# Обучаем базовую модель

Проверим загрузку видеокарты, прежде чем запустить обучение:

Запускаем обучение на 10 эпох

In [ ]:
loss_track, accuracy_track = trainval(model, loaders, optimizer, epochs=10)

 32%|███▏      | 34/105 [05:37<08:52,  7.50s/it]

In [ ]:
from matplotlib import pyplot as plt
%matplotlib inline
plt.plot(accuracy_track['training'], label='Training')
plt.plot(accuracy_track['validation'], label='Validation')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.grid()
plt.legend()

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

def predict_image(img, model):
    # Преобразование to a batch of 1
    xb = img.unsqueeze(0).to(device)
    # Получение прогнозов от модели
    yb = model(xb)
    # Выбираем индекс с наибольшей вероятностью
    _, preds  = torch.max(yb, dim=1)
    # Получение метки класса
    return dataset.classes[preds[0].item()]

for i in range(1,10):
  img, label = val_set[i]
  plt.imshow(img.clip(0,1).permute(1, 2, 0))
  plt.axis('off')
  plt.title('Label: {}, Predicted: {}'.format(dataset.classes[label],predict_image(img, model)))
  plt.show()

# Практическое задание



В заднии представлена логика выполнения с использованием tensorflow/keras. Выполнять можно как с использованием tensorflow/keras, так и pytorch.

1. Необходимо обучить предобученную сверточную архитектуру для задач классификации цветов.

В выбранной Вами архитектуре также необходимо **разобраться** с основными её параметрами и принципами работы.

Посмотрите как использовать [модели в PyTorch](https://pytorch.org/vision/stable/models.html) для классификации, выберите одну и используя transfer learning до-обучите модель на классификацию цветов. Чтобы это сделать замените ____ в ячейках ниже на работающий код.

2. Реализовать свою архитектуру, также как в разделе 'Сверточная нейросеть с нуля'.

3. Сравнить три архитектуры (из раздела 'Сверточная нейросеть с нуля', предобученую сверточную архитектуру и свою архитектуру (из п. 2)). Визуализировать полученный результат сравнения.





**Что такое transfer learning? Что такое предобученая нейронная сеть?**

**Предобученная нейронная сеть** — это модель, которая уже прошла процесс обучения на **большом и разнообразном наборе данных** (например, ImageNet, содержащем 14 миллионов изображений 1000 классов).

**Transfer Learning (передача обучения)** — это техника машинного обучения, при которой знания, полученные моделью при решении одной задачи, **переносятся** для решения другой, но связанной задачи.

**Как это работает на практике?**

1.   Берем предобученную модель (например, ResNet50 на ImageNet).
2.   **Замораживаем** большую часть ее слоев (обычно все, кроме последнего).
3.   **Заменяем** последний слой (классификатор) на новый, подходящий для нашей задачи (в нашем случае — 5 классов вместо 1000).
4.   **Дообучаем** (fine-tune) модель на нашем небольшом датасете. Модель использует свои мощные 'признаковые детекторы' для извлечения информации из наших изображений цветов, а новый классификатор учится интерпретировать эти признаки для нашей конкретной задачи.

1. Обучение предобученной сверточной архитектуры для задач классификации цветов

In [ ]:
# Импортируем предобученную модель ResNet50 из torchvision
# Параметр `weights` указывает, что мы хотим загрузить веса, предварительно обученные на датасете ImageNet
model = torchvision.models.resnet50(weights=torchvision.models.ResNet50_Weights.IMAGENET1K_V1)

**Добавьте описание архитектуры** выбранной Вами предобученой сверточной нейронной сети.

**ResNet50 (Residual Network 50)** — это глубокая сверточная нейронная сеть, состоящая из **50 слоев** (48 сверточных, 1 MaxPool и 1 Average Pool) . Ее ключевая особенность — использование **остаточных блоков (residual blocks)**. Вместо того чтобы напрямую изучать желаемое отображение `H(x)`, блок изучает **остаточную функцию** `F(x) = H(x) - x`, а затем добавляет вход `x` обратно к выходу: `H(x) = F(x) + x`. Эта 'skip-connection' (обходная связь) позволяет эффективно обучать очень глубокие сети, решая проблему исчезающих градиентов .

**Основные параметры:**
*   **Вход:** Изображения размером `224x224` пикселей с 3 цветовыми каналами (RGB).
*   **Выход:** Вектор из 1000 значений (для ImageNet), который мы заменяем на 5 для нашей задачи.
*   **Особенности:** Глубокая архитектура, остаточные связи, высокая точность в задачах классификации.

In [ ]:
# Функция для 'заморозки' всех слоев модели
# Это означает, что градиенты для этих слоев не будут вычисляться, и их веса не будут обновляться во время обучения
# Мы делаем это, чтобы сохранить знания, полученные моделью на ImageNet, и обучать только новый, последний слой
def set_parameter_requires_grad(model):
    for param in model.parameters(): # Проходим по всем параметрам (весам и смещениям) модели
        param.requires_grad = False  # Отключаем вычисление градиента для них

set_parameter_requires_grad(model) # Применяем функцию к нашей модели

**Что такое функция для заморозки весов модели?**

Функция `set_parameter_requires_grad` устанавливает атрибут `requires_grad` для всех параметров модели в значение `False`. В PyTorch этот атрибут определяет, нужно ли вычислять градиенты для данного параметра во время обратного распространения ошибки (backpropagation). Если `requires_grad=False`, градиенты не вычисляются, и веса не обновляются оптимизатором. Это позволяет 'заморозить' слои предобученной модели, сохраняя их полезные признаки, и обучать только новые слои (например, последний классификатор).


Последний слой (fc в ResNet) отвечает за классификацию на 1000 классов (ImageNet). Нам нужно заменить его на слой с 5 выходами (наши 5 видов цветов).

In [ ]:
# В ResNet50 последний слой называется `fc` (fully connected)
# По умолчанию он имеет 1000 выходов (для 1000 классов ImageNet)
# Нам нужно заменить его на новый линейный слой с 5 выходами, соответствующими нашим 5 классам цветов (тюльпан, ромашка и т.д.)
model.fc = nn.Linear(model.fc.in_features, 5) # `model.fc.in_features` автоматически дает нам количество входов в этот слой (2048 для ResNet50)

In [ ]:
# Выводим имена только тех параметров, для которых `requires_grad=True`
# После наших манипуляций мы должны увидеть только два параметра:
# `fc.weight` и `fc.bias` — веса и смещение нашего нового последнего слоя
# Все остальные параметры должны быть 'заморожены'
for name, param in model.named_parameters():
    if param.requires_grad:
        print(name)

Эта ячейка должна вывести только fc.weight и fc.bias. Это означает, что обучается только новый последний слой.

In [ ]:
# Отправляем модель на GPU для ускорения вычислений
model.to(device)

# Создаем оптимизатор. Важно передать в него `model.parameters()`
# Поскольку все параметры, кроме последнего слоя 'заморожены', оптимизатор будет обновлять только веса нового слоя
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Запускаем функцию обучения `trainval`, которую мы уже определили ранее
# Сохраняем историю потерь и точности в новые переменные, чтобы не перезаписать результаты базовой модели
loss_track_resnet, accuracy_track_resnet = trainval(model, loaders, optimizer, epochs=10)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
plt.plot(accuracy_track_resnet['training'], label='Training')
plt.plot(accuracy_track_resnet['validation'], label='Validation')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.grid()
plt.legend()

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

def predict_image(img, model):
    xb = img.unsqueeze(0).to(device)
    yb = model(xb)
    _, preds  = torch.max(yb, dim=1)
    return dataset.classes[preds[0].item()]

for i in range(1,10):
  img, label = val_set[i]
  plt.imshow(img.clip(0,1).permute(1, 2, 0))
  plt.axis('off')
  plt.title('Label: {}, Predicted: {}'.format(dataset.classes[label],predict_image(img, model)))
  plt.show()

По желанию, можно сохранить веса модели.

In [ ]:
weights_fname = '/content/drive/My Drive/Datas/ResNet50.pth'
torch.save(model.state_dict(), weights_fname)

2. Своя архитектура

Обновление подготовки данных (Аугментация)

Первым делом улучшим `prepare_imgs`, добавив аугментацию. Это искусственно увеличит датасет и сделает модель более устойчивой к вариациям.

In [ ]:
# Обновленный блок подготовки изображений с аугментацией для обучающей выборки
train_transform = torchvision.transforms.Compose([
    torchvision.transforms.Resize((256, 256)), # Сначала увеличиваем
    torchvision.transforms.RandomCrop(224),     # Затем случайно обрезаем до 224x224
    torchvision.transforms.RandomHorizontalFlip(p=0.5), # Случайное отражение по горизонтали
    torchvision.transforms.ColorJitter(brightness=0.2, contrast=0.2), # Небольшие изменения цвета
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Для валидации аугментация не нужна, только ресайз и нормализация
val_transform = torchvision.transforms.Compose([
    torchvision.transforms.Resize((224, 224)),
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

Теперь нужно пересоздать датасеты с новыми трансформациями:

In [ ]:
# Исходный датасет без трансформаций для разделения
base_dataset = ImageFolder('/content/drive/My Drive/Datas')

# Разделяем индексы
train_indices, val_indices = torch.utils.data.random_split(range(len(base_dataset)), [len(base_dataset)-1000, 1000])

# Создаем Subset с нужными трансформациями
from torch.utils.data import Subset
train_set = Subset(base_dataset, train_indices.indices)
val_set = Subset(base_dataset, val_indices.indices)

# Применяем трансформации
train_set.dataset.transform = train_transform
val_set.dataset.transform = val_transform

# Создаем загрузчики
loaders = {
    'training': DataLoader(train_set, batch_size=32, shuffle=True, num_workers=2, pin_memory=True),
    'validation': DataLoader(val_set, batch_size=32, shuffle=False, num_workers=2, pin_memory=True)
}

Эта модель использует **остаточные связи (Residual Connections)** внутри блоков, что позволяет ей быть глубже без риска деградации.

In [ ]:
class ResidualBlock(nn.Module):
    """
    Простой остаточный блок.
    Вход и выход должны иметь одинаковое количество каналов и пространственное разрешение.
    """
    def __init__(self, in_channels):
        super(ResidualBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, in_channels, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(in_channels)
        self.conv2 = nn.Conv2d(in_channels, in_channels, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(in_channels)
        self.relu = nn.ReLU(inplace=True)

    def forward(self, x):
        residual = x # Сохраняем исходный вход
        out = self.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += residual # Добавляем исходный вход к выходу сверток
        out = self.relu(out)
        return out


class EnhancedCNN(nn.Module):
    def __init__(self, num_classes=5):
        super(EnhancedCNN, self).__init__()

        self.features = nn.Sequential(
            # Начальный слой для увеличения количества каналов
            nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3), # Уменьшаем разрешение сразу
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2, padding=1), # Дополнительное уменьшение

            # Блок 1
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            ResidualBlock(128),
            ResidualBlock(128),
            nn.MaxPool2d(kernel_size=2, stride=2),

            # Блок 2
            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            ResidualBlock(256),
            ResidualBlock(256),
            nn.MaxPool2d(kernel_size=2, stride=2),

            # Блок 3 (самый глубокий)
            nn.Conv2d(256, 512, kernel_size=3, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(inplace=True),
            ResidualBlock(512),
            ResidualBlock(512),
            nn.AdaptiveAvgPool2d((4, 4)) # Фиксируем выходной размер для классификатора
        )

        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Dropout(0.5),
            nn.Linear(512 * 4 * 4, 1024),
            nn.ReLU(inplace=True),
            nn.Dropout(0.3), # Меньший Dropout перед последним слоем
            nn.Linear(1024, num_classes)
        )

    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x

Планировщик (`scheduler`) постепенно уменьшает скорость обучения, когда точность на валидации перестает улучшаться. Это помогает модели "дожать" последние проценты точности и избежать переобучения.

In [ ]:
# Создаем модель
enhanced_model = EnhancedCNN(num_classes=5).to(device)

# Оптимизатор и функция потерь
optimizer = torch.optim.Adam(enhanced_model.parameters(), lr=0.001, weight_decay=1e-4) # Добавляем L2-регуляризацию
criterion = nn.CrossEntropyLoss()

# Планировщик скорости обучения
from torch.optim.lr_scheduler import ReduceLROnPlateau
scheduler = ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=3)

# Функция обучения с поддержкой scheduler
def trainval_with_scheduler(model, loaders, optimizer, scheduler, epochs=10):
    best_val_acc = 0.0

    # Словари для хранения истории метрик по всем эпохам
    loss_track = {'training': [], 'validation': []}
    accuracy_track = {'training': [], 'validation': []}

    for epoch in range(epochs):
        for mode in ['training', 'validation']:
            # Словари для отслеживания метрик в текущей эпохе
            loss_meter = ValueMeter()
            accuracy_meter = ValueMeter()

            with torch.set_grad_enabled(mode == 'training'):
                model.train() if mode == 'training' else model.eval()

                for imgs, labels in tqdm(loaders[mode]):
                    imgs = imgs.to(device)
                    labels = labels.to(device)
                    bs = labels.shape[0]

                    # Forward pass
                    outputs = model(imgs)
                    loss = criterion(outputs, labels)
                    acc = accuracy(outputs, labels) # Используем уже определенную функцию accuracy

                    # Обновляем метрики
                    loss_meter.add(loss.item(), bs)
                    accuracy_meter.add(acc, bs)

                    if mode == 'training':
                        optimizer.zero_grad()
                        loss.backward()
                        optimizer.step()

            # Сохраняем метрики текущей эпохи
            epoch_loss = loss_meter.value()
            epoch_acc = accuracy_meter.value()
            loss_track[mode].append(epoch_loss)
            accuracy_track[mode].append(epoch_acc)

            # Выводим результаты для текущей фазы
            if mode == 'validation':
                # Обновление scheduler на основе точности валидации
                scheduler.step(epoch_acc)

            # Сохранение лучшей модели
            if epoch_acc > best_val_acc:
                best_val_acc = epoch_acc
                torch.save(model.state_dict(), '/content/drive/My Drive/Datas/best_enhanced_model.pth')

                # Выводим с указанием лучшей точности
                log(mode, epoch, loss_meter, accuracy_meter, best_perf=100 * best_val_acc)
            else:
                log(mode, epoch, loss_meter, accuracy_meter)

    return loss_track, accuracy_track, best_val_acc

# Запуск обучения
enhanced_model = EnhancedCNN(num_classes=5).to(device)
optimizer = torch.optim.Adam(enhanced_model.parameters(), lr=0.001, weight_decay=1e-4)
scheduler = ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=3)

# Обучаем и получаем историю
loss_track_enhanced, accuracy_track_enhanced, best_acc = trainval_with_scheduler(enhanced_model, loaders, optimizer, scheduler, epochs=10)

3. Сравнение и вузуализация 3-х архитектур

Теперь у нас есть три набора данных о точности:

'accuracy_track' — от базовой модели из начала ноутбука.

'accuracy_track_resnet' — от предобученной ResNet50.

'accuracy_track_custom' — от собственной архитектуры.

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
plt.figure(figsize=(12, 5))

# Левый график: Точность на обучающей выборке
plt.subplot(1, 2, 1)
# Строим кривые обучения для всех трех моделей
plt.plot(accuracy_track['training'], label='Базовая (train)', linewidth=2)
plt.plot(accuracy_track_resnet['training'], label='ResNet50 (train)', linewidth=2)
plt.plot(accuracy_track_enhanced['training'], label='Своя (train)', linewidth=2)
plt.title('Точность на обучающей выборке')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.grid(True)
plt.legend()

# Правый график: Точность на валидационной выборке
# Он показывает, насколько хорошо модель обобщает на новые данные
plt.subplot(1, 2, 2)
plt.plot(accuracy_track['validation'], label='Базовая (val)', linewidth=2)
plt.plot(accuracy_track_resnet['validation'], label='ResNet50 (val)', linewidth=2)
plt.plot(accuracy_track_enhanced['validation'], label='Своя Enhanced (val)', linewidth=2)
plt.title('Точность на валидационной выборке')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.grid(True)
plt.legend()

plt.tight_layout()
plt.show()

4.   Интерпретация результатов

## Вопросы.

**В чем основные отличия между сверточной нейронной сетью и 'обычной' полносвязной нейронной сетью?**

Основное отличие в том, **как они обрабатывают данные**.

**Полносвязная сеть (FCNN):** Каждый нейрон в слое соединен со **всеми** нейронами предыдущего слоя. Это приводит к огромному количеству параметров, особенно для изображений, и не учитывает пространственную локальность пикселей (соседние пиксели важнее дальних).

**Сверточная сеть (CNN):** Использует **сверточные слои**, где нейроны соединены только с небольшой локальной областью (ядром) предыдущего слоя. Это позволяет сети эффективно извлекать локальные признаки (например, края, текстуры) и обладает свойством **параметрической эффективности** (одни и те же веса ядра используются по всему изображению) и **эквивариантности к сдвигу** (признак будет обнаружен независимо от его положения на изображении).

**Простой пример:** Для изображения 224x224x3 (150,528 пикселей) первый полносвязный слой с 1024 нейронами имел бы 150,528 * 1024 ≈ 154 миллиона параметров! В CNN первый сверточный слой с ядром 3x3 и 64 фильтрами имеет всего 3 * 3 * 3 * 64 = 1,728 параметров.